In [5]:
a=[78.33,70.13,69.56,49.07,26.83]
b=[16229,27413,16096,4157,68167]
total=0.0
for i,num in enumerate(a):
    total+=num*b[i]
print (total/132060)

48.05568393154626


In [1]:
import transformers
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [2]:
%matplotlib notebook
%matplotlib inline

VQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/VQA'
GQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/GQA'

import pandas as pd
import json
import pickle as pkl
import numpy as np
import h5py
import random
from PIL import Image
import IPython.display as display
from collections import defaultdict
import os

import matplotlib.pyplot as plt
import torch

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [3]:
CUDA_DEVICE=15
torch.cuda.set_device(CUDA_DEVICE)
device = torch.device("cuda:"+str(CUDA_DEVICE))
#the default ipykernel links to the first conda environment

In [4]:
t5_model = T5ForConditionalGeneration.from_pretrained('t5-large')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-large')

/root/anaconda3/envs/nlp-rui/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [5]:
t5_tokenizer.sep_token = '</s>'
t5_model = t5_model.to(device)
t5_model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (

In [6]:
special_token_mapping = {'cls': t5_tokenizer.cls_token_id,
                         'mask': t5_tokenizer.mask_token_id,
                         'sep': t5_tokenizer.sep_token_id, 
                         'sep+': t5_tokenizer.sep_token_id}

In [7]:
for i in range(10):
    special_token_mapping["<extra_id_%d>" % (i)] = \
    t5_tokenizer._convert_token_to_id("<extra_id_%d>" % (i))
for w in special_token_mapping:
    print (w,special_token_mapping[w])

cls None
mask None
sep 1
sep+ 1
<extra_id_0> 32099
<extra_id_1> 32098
<extra_id_2> 32097
<extra_id_3> 32096
<extra_id_4> 32095
<extra_id_5> 32094
<extra_id_6> 32093
<extra_id_7> 32092
<extra_id_8> 32091
<extra_id_9> 32090


In [7]:
def enc(text):
    return t5_tokenizer.encode(text, add_special_tokens=False)

In [ ]:
input_text='the <extra_id_0> is on the man\'s back.'
tokens=enc(input_text)
print (tokens)

In [8]:
ENC_MAX_LEN=20
DEC_MAX_LEN=20

In [ ]:
input_tensors = [torch.tensor(tokens).long()]
max_length = max(ENC_MAX_LEN, len(tokens))

In [ ]:
input_ids = torch.zeros((len(input_tensors), max_length)).long()
attention_mask = torch.zeros((len(input_tensors), max_length)).long()

In [ ]:
for i in range(len(input_tensors)):
    input_ids[i, :input_tensors[i].size(-1)] = input_tensors[i]
    attention_mask[i, :input_tensors[i].size(-1)] = 1

In [ ]:
print(t5_tokenizer.decode(input_ids[0]))
print (input_ids[0])

In [ ]:
input_ids = input_ids.cuda()
attention_mask = attention_mask.cuda()

In [ ]:
start_mask = t5_tokenizer._convert_token_to_id('<extra_id_0>')
ori_decoder_input_ids = torch.zeros((input_ids.size(0), DEC_MAX_LEN)).long()
ori_decoder_input_ids[..., 0] = t5_model.config.decoder_start_token_id

In [ ]:
current_output = [{'decoder_input_ids': ori_decoder_input_ids,
                   'll': 0, 'output_id': 1, 'output': [], 
                   'last_length': -1}]

In [ ]:
beam=5
target_number=2
for i in range(max_length - 2):
    print ('Current len:',i)
    new_current_output = []
    for item in current_output:
        print ('\tLength of current_output',len(current_output))
        if item['output_id'] > target_number:
            # Enough contents
            new_current_output.append(item)
            continue
        decoder_input_ids = item['decoder_input_ids']

        # Forward
        batch_size = 32
        turn = input_ids.size(0) // batch_size
        if input_ids.size(0) % batch_size != 0:
            turn += 1
        aggr_output = []
        for t in range(turn):
            start = t * batch_size
            end = min((t + 1) * batch_size, input_ids.size(0))

            with torch.no_grad():
                aggr_output.append(t5_model(
                    input_ids[start:end], 
                    attention_mask=attention_mask[start:end], 
                    decoder_input_ids=decoder_input_ids.cuda()[start:end])[0])
        aggr_output = torch.cat(aggr_output, 0)

        # Gather results across all input sentences, and sort generated tokens by log likelihood
        aggr_output = aggr_output.mean(0)
        log_denominator = torch.logsumexp(aggr_output[i], -1).item()
        ids = list(range(t5_model.config.vocab_size))
        ids.sort(key=lambda x: aggr_output[i][x].item(), reverse=True)
        ids = ids[:beam+3]
            
        for k,word_id in enumerate(ids):
            #print (k,'-th word')
            output_id = item['output_id']

            if word_id == start_mask - output_id or \
            word_id == t5_tokenizer._convert_token_to_id('</s>'):
                # Finish one part
                check = True
                output_id += 1
                last_length = 0
            else:
                last_length = item['last_length'] + 1
                check = True

            output_text = item['output'] + [word_id]
            ll = item['ll'] + aggr_output[i][word_id] - log_denominator
            new_decoder_input_ids = decoder_input_ids.new_zeros(decoder_input_ids.size())
            new_decoder_input_ids[:] = decoder_input_ids
            new_decoder_input_ids[..., i + 1] = word_id
            
            if word_id in [3, 19794, 22354]:
                check = False

            # Forbid continuous "."
            if len(output_text) > 1 and output_text[-2] == 5 and output_text[-1] == 5:
                check = False

            if check:
                # Add new results to beam search pool
                new_item = {'decoder_input_ids': new_decoder_input_ids, 'll': ll, 'output_id': output_id, 'output': output_text, 'last_length': last_length}
                new_current_output.append(new_item)
    print ('One iteration finished...')
    if len(new_current_output) == 0:
        break

    new_current_output.sort(key=lambda x: x['ll'], reverse=True)
    new_current_output = new_current_output[:beam]
    current_output = new_current_output

In [ ]:
result = []
print("####### generated results #######")
for item in current_output:
    generate_text = ''
    for token in item['output']:
        generate_text += t5_tokenizer._convert_id_to_token(token)
    print('--------------')
    print('score:', item['ll'].item())
    print('generated ids', item['output'])
    print('generated text', generate_text)
    print (len(generate_text))
    result.append(generate_text)
print("####### generated results #######\n",result)

In [ ]:
text = 'the <extra_id_0> is on the wall on the left.'

encoded = t5_tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt')
input_ids = encoded['input_ids'].to(device)

# Generaing 20 sequences with maximum length set to 5
outputs = t5_model.generate(input_ids=input_ids, 
                            num_beams=100, num_return_sequences=100,
                            max_length=30)

_0_index = text.index('<extra_id_0>')
_result_prefix = text[:_0_index]
_result_suffix = text[_0_index+20:]  # 12 is the length of <extra_id_0>

def _filter(output, end_token='<extra_id_1>'):
    # The first token is <unk> (inidex at 0) and the second token is <extra_id_0> (indexed at 32099)
    _txt = t5_tokenizer.decode(output[2:], skip_special_tokens=False, clean_up_tokenization_spaces=False)
    #print (_txt)
    text=_txt.replace('<extra_id_1>',' <s>')
    #print (text)
    text=text.replace('.',' <s>')
    text=text.replace(',',' <s>')
    result=text.split('<s>')[0].strip()
    if '\'s' in result:
        result=result.replace('\'s',' <s>')
        #print (result)
        return result.split('<s>')[1].strip()
    return text.split('<s>')[0].strip()
    """if end_token in _txt:
        _end_token_index = _txt.index(end_token)
        return _result_prefix + _txt[:_end_token_index] + _result_suffix
    else:
        return _result_prefix + _txt + _result_suffix"""

results = list(map(_filter, outputs))

In [9]:
def candidate_generator(text):
    encoded = t5_tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt')
    input_ids = encoded['input_ids'].to(device)

    # Generaing 20 sequences with maximum length set to 5
    outputs = t5_model.generate(input_ids=input_ids, 
                                num_beams=50, num_return_sequences=30,
                                max_length=30)
    results = list(map(_filter, outputs))
    results=list(set(results))
    return results

In [8]:
val_all=load_json(os.path.join(VQA_PATH,'scenes','val_all.json'))
print ('Length of testing data:',len(val_all))
q_types_dict=defaultdict(int)
for row in val_all:
    q_types_dict[row['q_type']]+=1
print (q_types_dict)
val_based_qt={q_t:[] for q_t in q_types_dict}
for row in val_all:
    val_based_qt[row['q_type']].append(row)

Length of testing data: 214354
defaultdict(<class 'int'>, {'none of the above': 8550, 'what are the': 3282, 'what is': 6328, 'what': 15897, 'is this a': 7492, 'is this': 7841, 'what is the man': 2663, 'how many': 20462, 'what does the': 1970, 'why': 1438, 'is it': 3566, 'why is the': 514, 'what color is the': 14061, 'is there a': 4679, 'is the': 17265, 'is that a': 714, 'are these': 2839, 'are the': 5264, 'what is the': 11353, 'which': 2448, 'could': 618, 'are there': 2771, 'what kind of': 5840, 'has': 946, 'what color are the': 3118, 'are there any': 1330, 'is this person': 734, 'does the': 3183, 'where is the': 3716, 'how many people are': 2005, 'can you': 872, 'what type of': 4040, 'what is the color of the': 826, 'what is on the': 2174, 'does this': 2227, 'is this an': 890, 'who is': 1070, 'what is this': 1696, 'is there': 3120, 'are': 2359, 'what time': 1746, 'how': 2422, 'what is in the': 1733, 'do you': 724, 'what are': 1556, 'what sport is': 1086, 'what is the person': 900, 'is

In [9]:
qid_to_t5={}
with open('../clip_vqa/other_filtered_T5_large.json') as other:
    for line in other:
        data = json.loads(line.strip())
        qid_to_t5[data['qid']] = data
print (len(qid_to_t5))

105600


In [10]:
def generate_statement(statement,ques):
    sent=[]
    
    if 'doing' in ques[:-1]:
        inplace='action'
    elif 'color' in ques[:-1]:
        inplace='color'
    elif 'made of' in ques[:-1]:
        inplace='material'
    else:
        inplace='item'
            
    flag=True
    for w in statement.split(' '):
        if w==inplace:
            if flag:
                sent.append('<extra_id_0>')
                flag=False
        else:
            sent.append(w)
    if flag:
        sent.append('<extra_id_0>')
    sent=' '.join(sent)+'.'
    return sent

In [11]:
def verify_ans(gt,pred):
    flag=False
    in_list=[]
    for w in gt:
        if w in pred:
            flag=True
            in_list.append(w)
    return flag,in_list

In [12]:
def _filter(output, end_token='<extra_id_1>'):
    # The first token is <unk> (inidex at 0) and the second token is <extra_id_0> (indexed at 32099)
    _txt = t5_tokenizer.decode(output[2:], skip_special_tokens=False, clean_up_tokenization_spaces=False)
    #print (_txt)
    text=_txt.replace('<extra_id_1>',' <s>')
    #print (text)
    text=text.replace('.',' <s>')
    text=text.replace(',',' <s>')
    result=text.split('<s>')[0].strip()
    if '\'s' in result:
        result=result.replace('\'s',' <s>')
        #print (result)
        return result.split('<s>')[1].strip()
    return text.split('<s>')[0].strip()

In [13]:
import enchant
en_dict=enchant.Dict("en_US")

In [14]:
from nltk.corpus import stopwords
removed_words = set(stopwords.words('english'))

In [15]:
removed_words=list(removed_words)
removed_words.extend(['am','is','are','was','were'])

In [16]:
def process(words):
    results=[]
    for w in words:
        if len(w)<=1:
            continue
        if w in ['""',',','“','.','?',':','??','(',')','-','/']:
            continue
        if w in ['<unk>','[unk]','unk']:
            continue
        if 'the' in w.split(' '):
            w_list=[s for s in w.split(' ') if s !='the']
            w=' '.join(w_list).lower()
        elif 'this' in w.split(' '):
            w_list=[s for s in w.split(' ') if s !='this']
            w=' '.join(w_list).lower()
        if w in removed_words:
            continue
        if w.isdigit():
            continue
        if len(w)>1 and en_dict.check(w):
            results.append(w.lower())
    return results

In [17]:
def candidate_generator(text):
    encoded = t5_tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt')
    input_ids = encoded['input_ids'].to(device)

    # Generaing 20 sequences with maximum length set to 5
    outputs = t5_model.generate(input_ids=input_ids, 
                                num_beams=1000, num_return_sequences=350,
                                max_length=3)
    results = list(map(_filter, outputs))
    results=list(set(results))
    results=process(results)
    results=list(set(results))
    return results

In [18]:
text='the two individuals riding is <extra_id_0>'
results=candidate_generator(text)
print (results)

2022-09-29 09:04:12.493921: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-29 09:04:12.493975: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


['striking', 'encouraging', 'one', 'unchanged', 'poor', 'reasonable', 'shown', 'successful', 'beyond', 'outstanding', 'three', 'disappointing', 'minimal', 'insane', 'awkward', 'another', 'useless', 'like', 'normal', 'interesting', 'common', 'lacking', 'good', 'healthy', 'really', 'satisfactory', 'recommended', 'happening', 'stronger', 'allowed', 'ideal', 'ridiculous', 'included', 'disturbing', 'considered', 'stunning', 'free', 'unknown', 'frightening', 'worse', 'essential', 'significant', 'complicated', 'vital', 'suspicious', 'equal', 'prohibited', 'unexpected', 'extremely', 'identical', 'relevant', 'required', 'fascinating', 'illegal', 'achieved', 'clear', 'hard', 'inaccurate', 'satisfied', 'uncertain', 'huge', 'known', 'coming', 'superior', 'mentioned', 'close', 'strong', 'impressive', 'stated', 'unlikely', 'embarrassing', 'useful', 'confirmed', 'explained', 'involved', 'concerned', 'completed', 'confidential', 'closed', 'shocking', 'quite', 'terrifying', 'incomplete', 'used', 'param

In [24]:
q_t='what is the'
vis=0
upper=0.0
showing=True
random.shuffle(val_based_qt[q_t])
acc=0.0
upper_acc=0.0
print(len(val_based_qt[q_t]))
for row in val_based_qt[q_t]:
    vis+=1
    if vis>10:
        break
    if row['ans_type']!='other':
        continue
    if row['idx'] not in qid_to_t5:
        continue
        
    gt_scores=row['scores']
    answers=row['answer']
    ques=row['question']
    
    base_info=qid_to_t5[row['idx']]
    statement=base_info['prompts'][0]
    t5_pred=candidate_generator(statement)
    
    upper,in_list=verify_ans(row['answer'],t5_pred)
    if upper:
        upper_acc+=1
        
    print (ques,answers,row['img_id'])
    print ('\t',statement)
    print ('\tMine: ',len(t5_pred),upper,in_list)
    upper,in_list=verify_ans(row['answer'],base_info['labels'])
    print ('\tOthers: ',len(base_info['labels']),upper,in_list)
    print (
        #t5_pred,
        '\n')
    img_id='COCO_val2014_'+str(row['img_id']).zfill(12)+'.jpg'
    if vis%200==0:
        print ('Upper acc',upper_acc*100.0/vis)
print ('Total:',vis,'\n\tUpper acc',upper_acc*100.0/vis)

11353
What is the colorful object? ['kite', 'yes', 'parachute'] 104796
	 the colorful object is <extra_id_0> .
	Mine:  308 False []
	Others:  18 False []


What is the purpose of the glasses worn by the man? ['safety', 'protection'] 449895
	 the purpose of the glasses worn by the man is <extra_id_0> .
	Mine:  292 True ['safety']
	Others:  161 True ['safety', 'protection']


What is the person's gender? ['male'] 127669
	 the person's gender is <extra_id_0> .
	Mine:  258 True ['male']
	Others:  142 True ['male']


What is the woman's holding? ['phone', 'umbrella', 'umbrella, camera'] 235017
	 the woman's holding is <extra_id_0> .
	Mine:  299 False []
	Others:  137 True ['umbrella']


What is the an wearing? ['sweater', '2 shirts', 'striped shirt', 'striped sweater', 'shirt'] 450282
	 the an wearing is <extra_id_0> .
	Mine:  268 False []
	Others:  166 False []


What is the statue holding? ['walking cane', 'stuffed animal', 'cane', 'stick', 'walking stick', 'gun', 'teddy bear'] 209763
	 t

In [26]:
upper_their=0.0
upper_mine=0.0
vis=0
total={}
random.shuffle(val_all)
invalid=[]
for row in val_all:
    if row['ans_type']!='other':
        continue
    if row['idx'] not in qid_to_t5:
            continue
    vis+=1
    #if vis>30:
    #    break
    answers=row['answer']
    ques=row['question']
    
    base_info=qid_to_t5[row['idx']]
    statement=base_info['prompts'][0]
    try:
        t5_pred=candidate_generator(statement)
    except:
        invalid.append(row['idx'])
        print ('Invalid!!!!',row['idx'])
        continue
        
    if len(base_info['labels'])<80:
        t5_pred=base_info['labels']
    
    if verify_ans(row['answer'],t5_pred)[0]:
        upper_mine+=1
    if verify_ans(row['answer'],base_info['labels'])[0]:
        upper_their+=1
    
    if vis%500==0:
        print ('Already finished:',vis,len(total)+1)
        print ('\tMine acc:',100.0*upper_mine/vis)
        print ('\tTheir acc:',100.0*upper_their/vis)
    total[row['idx']]=t5_pred
    #json.dump(t5_pred,
    #          open(os.path.join('mine_t5_file',row['idx']+'.json'),'w'))

Already finished: 500 500
	Mine acc: 62.2
	Their acc: 83.4
Already finished: 1000 1000
	Mine acc: 62.8
	Their acc: 84.3
Already finished: 1500 1500
	Mine acc: 63.6
	Their acc: 85.4
Already finished: 2000 2000
	Mine acc: 63.15
	Their acc: 85.3
Already finished: 2500 2500
	Mine acc: 63.56
	Their acc: 85.68
Already finished: 3000 3000
	Mine acc: 63.6
	Their acc: 85.6
Already finished: 3500 3500
	Mine acc: 63.4
	Their acc: 85.45714285714286
Already finished: 4000 4000
	Mine acc: 62.925
	Their acc: 85.375
Already finished: 4500 4500
	Mine acc: 63.2
	Their acc: 85.62222222222222
Already finished: 5000 5000
	Mine acc: 62.78
	Their acc: 85.44
Already finished: 5500 5500
	Mine acc: 62.763636363636365
	Their acc: 85.41818181818182
Already finished: 6000 6000
	Mine acc: 62.916666666666664
	Their acc: 85.76666666666667
Already finished: 6500 6500
	Mine acc: 62.95384615384615
	Their acc: 85.8923076923077
Already finished: 7000 7000
	Mine acc: 62.628571428571426
	Their acc: 85.88571428571429
Already

Already finished: 50500 50500
	Mine acc: 62.883168316831686
	Their acc: 85.59207920792079
Already finished: 51000 51000
	Mine acc: 62.89411764705882
	Their acc: 85.58039215686274
Already finished: 51500 51500
	Mine acc: 62.91067961165049
	Their acc: 85.57281553398059
Already finished: 52000 52000
	Mine acc: 62.9
	Their acc: 85.57692307692308
Already finished: 52500 52500
	Mine acc: 62.89904761904762
	Their acc: 85.54857142857144
Already finished: 53000 53000
	Mine acc: 62.90188679245283
	Their acc: 85.53018867924528
Already finished: 53500 53500
	Mine acc: 62.904672897196264
	Their acc: 85.53084112149533
Already finished: 54000 54000
	Mine acc: 62.912962962962965
	Their acc: 85.52592592592593
Already finished: 54500 54500
	Mine acc: 62.88073394495413
	Their acc: 85.5137614678899
Already finished: 55000 55000
	Mine acc: 62.876363636363635
	Their acc: 85.52
Already finished: 55500 55500
	Mine acc: 62.863063063063066
	Their acc: 85.52072072072072
Already finished: 56000 56000
	Mine acc: 6

Already finished: 97500 97500
	Mine acc: 62.91897435897436
	Their acc: 85.60205128205128
Already finished: 98000 98000
	Mine acc: 62.92857142857143
	Their acc: 85.60306122448979
Already finished: 98500 98500
	Mine acc: 62.93502538071066
	Their acc: 85.60812182741117
Already finished: 99000 99000
	Mine acc: 62.93131313131313
	Their acc: 85.60808080808081
Already finished: 99500 99500
	Mine acc: 62.94070351758794
	Their acc: 85.62110552763819
Already finished: 100000 100000
	Mine acc: 62.949
	Their acc: 85.619
Already finished: 100500 100500
	Mine acc: 62.94726368159204
	Their acc: 85.62388059701493
Already finished: 101000 101000
	Mine acc: 62.95247524752475
	Their acc: 85.63168316831683
Already finished: 101500 101500
	Mine acc: 62.95566502463054
	Their acc: 85.63448275862069
Already finished: 102000 102000
	Mine acc: 62.95392156862745
	Their acc: 85.63627450980393
Already finished: 102500 102500
	Mine acc: 62.93853658536585
	Their acc: 85.61853658536586
Already finished: 103000 103000

In [27]:
pkl.dump(total,open('mine_t5_file/open_ended.pkl','wb'))

In [ ]:
q_types=[
    'what are the','what is','what is the man','what does the',
    ###'why','why is the','how',
    ##'which','what kind of','what is the','what type of','what',
    ##'who is','what is this','what sport is','what room is','what animal is',
    'which','what kind of','what is the','what type of','what',
    'who is','what is this','what sport is','what room is','what animal is',
    
    'where is the','what is on the',
    'what time','what is in the','what are',
    'what is the person','where are the',
    #'what brand',
    'what is the woman',
    'what is the name']

In [ ]:
#q_types=['what is the woman','what is on the']
for q_t in q_types:
    vis=0
    upper=0.0
    bart_results=load_pkl(os.path.join('bart_vqa',q_t+'.pkl'))
    statement_results=load_pkl(os.path.join('../qanli',q_t+'.pkl'))
    random.shuffle(val_based_qt[q_t])
    upper_acc=0.0
    print(q_t,len(val_based_qt[q_t]))
    save_pred={}
    for row in val_based_qt[q_t]:
        vis+=1
        answers=row['answer']
        ques=row['question']
    
        statement=generate_statement(statement_results[row['idx']],
                                     row['question'])
        try:
            t5_pred=candidate_generator(statement)
        except:
            t5_pred=bart_results[row['idx']]
        #t5_pred=candidate_generator(statement)
            
        save_pred[row['idx']]=t5_pred
        upper,in_list=verify_ans(row['answer'],t5_pred)
        if upper:
            upper_acc+=1
        if vis%200==0:
            print ('Upper acc',upper_acc*100.0/vis)
    print ('Total:',vis,'\n\tUpper acc',upper_acc*100.0/vis)
    pkl.dump(save_pred,open(os.path.join('t5_vqa',q_t+'.pkl'),'wb'))

In [19]:
val_all=json.load(
    open(os.path.join(GQA_PATH,'original','testdev_balanced_questions.json')
         ,'r')
)
print (len(val_all))
names=list(val_all.keys())

12578


In [13]:
val_all=load_json(os.path.join(GQA_PATH,'questions','val_all_data.json'))
print ('Length of testing data:',len(val_all))
print (val_all[0].keys(),'\n',val_all[0])

Length of testing data: 132062
dict_keys(['question', 'answer', 'img', 'layout', 'key', 'width', 'height', 'full_ans', 'types']) 
 {'question': 'What is this bird called?', 'answer': 'parrot', 'img': '2405722', 'layout': [{'operation': 'select', 'dependencies': [], 'argument': 'bird (329774)'}, {'operation': 'query', 'dependencies': [0], 'argument': 'name'}], 'key': '05515938', 'width': 500, 'height': 375, 'full_ans': 'This is a parrot.', 'types': {'detailed': 'categoryThis', 'semantic': 'cat', 'structural': 'query'}}


In [20]:
q_types_dict=defaultdict(int)
for name in val_all:
    row=val_all[name]
    q_types_dict[row['types']['structural']]+=1
print (q_types_dict)
val_based_qt={q_t:[] for q_t in q_types_dict}
for name in val_all:
    row=val_all[name]
    row['key']=name
    val_based_qt[row['types']['structural']].append(row)

defaultdict(<class 'int'>, {'verify': 2252, 'query': 6805, 'logical': 1803, 'choose': 1129, 'compare': 589})


In [14]:
q_types_dict=defaultdict(int)
for row in val_all:
    q_types_dict[row['types']['structural']]+=1
print (q_types_dict)
val_based_qt={q_t:[] for q_t in q_types_dict}
for row in val_all:
    val_based_qt[row['types']['structural']].append(row)

defaultdict(<class 'int'>, {'query': 68167, 'choose': 16229, 'verify': 27413, 'compare': 4157, 'logical': 16096})


In [29]:
temp_file=load_pkl('../GQA_ques_convertor/query_updates.pkl')
print(len(temp_file))
names=list(temp_file.keys())
print (temp_file[names[0]])

68167


In [21]:
temp_file=load_pkl('../GQA_ques_convertor/dev_query.pkl')
print(len(temp_file))
names=list(temp_file.keys())
print (temp_file[names[0]])

6805
Item is wearing the dress.


In [37]:
vis=0
invalid=0
random.shuffle(val_based_qt['query'])
for row in val_based_qt['query']:
    vis+=1
    if type(temp_file[row['key']]) is not str:
        temp_file[row['key']]=row['full_ans'].lower().replace(row['answer'],'item')
        if invalid<20:
            print (row['question'])
            print ('\t',temp_file[row['key']])
        invalid+=1
print (invalid)
    #print (row['question'])
    #print ('\t',temp_file[row['key']])

0


In [35]:
pkl.dump(temp_file,open('../GQA_ques_convertor/query_updates.pkl','wb'))

In [22]:
vis=0
random.shuffle(val_based_qt['query'])
for row in val_based_qt['query']:
    if vis>10:
        break
    vis+=1
    print (row['question'])
    print ('\t',temp_file[row['key']])

Which shape is the soccer ball that is above the tree?
	 The soccer ball that is above the tree is item.
What are the flowers in front of?
	 The flowers in front of are item.
What item of furniture is sitting atop the floor?
	 Item is sitting atop the floor.
What color is the picture?
	 The picture is item.
Who is sitting?
	 Item is sitting.
What is the bison doing?
	 The bison is doing item.
What's attached to the coat?
	 Item's attached to the coat.
What is leaning against the window?
	 Item is leaning against the window.
What animal is the male person carrying?
	 The male person is carrying item.
What's the belt holding onto?
	 The belt's holding onto item.
What is the vehicle that the fence is in front of called?
	 The vehicle that the fence is in front of called is item.


In [6]:
from transformers import BartTokenizer, BartForConditionalGeneration

bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

In [7]:
CUDA_DEVICE=1
torch.cuda.set_device(CUDA_DEVICE)
device = torch.device("cuda:"+str(CUDA_DEVICE))

In [8]:
bart_model=bart_model.to(device)

In [23]:
def generate_statement(sent):
    flag=True
    words=[]
    for w in sent[:-1].lower().split(' '):
        if w =='item' and flag:
            flag=False
            words.append('<extra_id_0>')
        else:
            words.append(w)
    words.append('.')
    return ' '.join(words)

In [25]:
q_t='query'
vis=0
upper=0.0
showing=True
random.shuffle(val_based_qt[q_t])
acc=0.0
upper_acc=0.0
print(len(val_based_qt[q_t]))
for row in val_based_qt[q_t]:
    vis+=1
    if vis>10:
        break
        
    answers=row['answer']
    ques=row['question']
    
    statement=generate_statement(temp_file[row['key']])
    t5_pred=candidate_generator(statement)
    
    upper,in_list=verify_ans([row['answer']],t5_pred)
    if upper:
        upper_acc+=1
        
    print (ques,answers,row['imageId'])
    print ('\t',statement)
    print ('\tMine: ',len(t5_pred),upper,in_list)
    print (
        t5_pred,
        '\n')
    if vis%200==0:
        print ('Upper acc',upper_acc*100.0/vis)
print ('Total:',vis,'\n\tUpper acc',upper_acc*100.0/vis)

6805
Which kind of animal isn't short? goat n187961
	 n't short is <extra_id_0> .
	Mine:  211 False []
['hi', 'big', 'impossible', 'missing', 'poor', 'valid', 'like', 'accurate', 'said', 'new', 'another', 'well', 'free', 'cool', 'less', 'already', 'getting', 'ha', 'great', 'first', 'full', 'known', 'anyone', 'possible', 'limited', 'necessary', 'total', 'fast', 'meaning', 'correct', 'wonderful', 'middle', 'best', 'need', 'beautiful', 'available', 'stupid', 'word', 'fat', 'co', 'thin', 'used', 'usually', 'many', 'longer', 'coming', 'shorter', 'top', 'difficult', 'meant', 'anyway', 'lost', 'nice', 'even', 'mid', 'completely', 'dead', 'life', 'nu', 'complete', 'miss', 'done', 'words', 'whole', 'fair', 'la', 'fine', 'bad', 'ca', 'man', 'yet', 'tough', 'go', 'boring', 'enough', 'always', 'safe', 'bit', 'also', 'interesting', 'must', 'going', 'fun', 'short', 'rather', 'non', 'last', 'us', 'truth', 'important', 'back', 'without', 'tall', 'cute', 'really', 'low', 'fake', 'high', 'red', 'made', 

What animals are shown in the photo? elephants n543966
	 <extra_id_0> are shown in the photo .
	Mine:  251 False []
['tips', 'sizes', 'key', 'differences', 'configuration', 'pictures', 'type', 'connections', 'like', 'working', 'plates', 'look', 'several', 'specs', 'diagram', 'windows', 'cups', 'needs', 'new', 'exact', 'well', 'aspects', 'pieces', 'glass', 'connection', 'product', 'designs', 'colors', 'numbers', 'first', 'full', 'others', 'finished', 'arrangements', 'unit', 'rules', 'material', 'order', 'frames', 'information', 'control', 'steps', 'best', 'available', 'beautiful', 'parameters', 'structure', 'test', 'version', 'letters', 'size', 'used', 'many', 'elements', 'spring', 'lamp', 'top', 'conditions', 'drawings', 'tray', 'versions', 'specific', 'combination', 'tools', 'slides', 'fit', 'button', 'dimensions', 'space', 'provided', 'area', 'three', 'photo', 'problems', 'styles', 'logo', 'words', 'attachment', 'dimension', 'fitting', 'details', 'image', 'requirements', 'holes', 'ti

What is the appliance that is to the right of the tap called? refrigerator n64959
	 the appliance that is to the right of the tap called is <extra_id_0> .
	Mine:  209 False []
['dishwasher', 'commonly', 'selected', 'drain', 'double', 'pressure', 'type', 'label', 'like', 'automatic', 'working', 'heater', 'boiling', 'remote', 'said', 'thermostat', 'considered', 'another', 'well', 'put', 'installed', 'free', 'operated', 'already', 'described', 'tap', 'typically', 'the', 'looking', 'finished', 'known', 'unit', 'necessary', 'mixer', 'flush', 'plug', 'connected', 'best', 'available', 'phone', 'fixed', 'used', 'hidden', 'lets', 'turning', 'usually', 'fridge', 'entitled', 'responsible', 'broken', 'coming', 'meant', 'heat', 'marked', 'essential', 'therefore', 'open', 'disconnected', 'completely', 'appliance', 'opposite', 'provided', 'taken', 'named', 'currently', 'pulled', 'fuse', 'done', 'to', 'call', 'see', 'fitting', 'replaced', 'la', 'ideal', 'using', 'temperature', 'situated', 'always', 's

In [27]:
def generate_statement_bart(sent):
    flag=True
    words=[]
    for w in sent[:-1].lower().split(' '):
        if w =='item' and flag:
            flag=False
            words.append('the')
            words.append('<mask>')
        else:
            words.append(w)
    if '<mask>' not in words:
        words.append('<mask>')
    words.append('.')
    return ' '.join(words)

In [28]:
def generate_statement_gt(sent,ans):
    flag=True
    words=[]
    for w in sent[:-1].lower().split(' '):
        if w ==ans and flag:
            flag=False
            words.append('the')
            words.append('<mask>')
        else:
            words.append(w)
    words.append('.')
    return ' '.join(words)

In [29]:
def candidate_generator_bart(statement):
    input_ids = bart_tokenizer([statement],
                               return_tensors="pt")["input_ids"]
    logits = bart_model(input_ids.to(device)).logits
        #print ((input_ids[0] == bart_tokenizer.mask_token_id).sum())
    masked_index = (input_ids[0] == bart_tokenizer.mask_token_id).nonzero().item()
    
    probs = logits[0, masked_index].softmax(dim=0)
    values, predictions = probs.topk(200)
    pred=bart_tokenizer.decode(predictions).split()
    results=list(set(pred))
    results=process(results)
    results=list(set(results))
    return results

In [35]:
q_t='query'
vis=0
upper=0.0
showing=True
random.shuffle(val_based_qt[q_t])
acc=0.0
upper_acc=0.0
print ('T5 generated with generated template')
total={}
print(len(val_based_qt[q_t]))
for row in val_based_qt[q_t]:
    #if vis>10:
    #    break
    vis+=1
        
    answers=row['answer']
    ques=row['question']
    #print (temp_file[row['key']])
    try:
        statement=generate_statement_bart(temp_file[row['key']])
    except:
        statement=generate_statement_gt(row['fullAnswer'],answers)
    statement=statement.replace('<mask>',
                                '<extra_id_0>')
    t5_pred=candidate_generator(statement)
    
    upper,in_list=verify_ans([row['answer']],t5_pred)
    if upper:
        upper_acc+=1
    total[row['key']]=t5_pred    
    """print (ques,answers,row['img'])
    print ('\t',statement)
    print ('\tMine: ',len(t5_pred),upper,in_list)
    print (
        t5_pred,
        '\n')"""
    if vis%2000==0:
        print ('Upper acc',upper_acc*100.0/vis)
print ('Total:',vis,'\n\tUpper acc',upper_acc*100.0/vis)
#pkl.dump(total,open('gqa_t5_pred/t5_generated_temp.pkl','wb'))

T5 generated with generated template
6805
Upper acc 52.8
Upper acc 53.075
Upper acc 53.45
Total: 6805 
	Upper acc 53.37252020573108


In [36]:
pkl.dump(total,open('gqa_t5_pred/dev_t5_generated_temp.pkl','wb'))

In [26]:
q_t='query'
vis=0
upper=0.0
showing=True
random.shuffle(val_based_qt[q_t])
acc=0.0
upper_acc=0.0
print ('T5 generated with generated template')
total={}
print(len(val_based_qt[q_t]))
for row in val_based_qt[q_t]:
    vis+=1
        
    answers=row['answer']
    ques=row['question']
    
    statement=generate_statement_bart(temp_file[row['key']]).replace('<mask>',
                                                                     '<extra_id_0>')
    t5_pred=candidate_generator(statement)
    
    upper,in_list=verify_ans([row['answer']],t5_pred)
    if upper:
        upper_acc+=1
    total[row['key']]=t5_pred    
    """print (ques,answers,row['img'])
    print ('\t',statement)
    print ('\tMine: ',len(t5_pred),upper,in_list)
    print (
        t5_pred,
        '\n')"""
    if vis%2000==0:
        print ('Upper acc',upper_acc*100.0/vis)
print ('Total:',vis,'\n\tUpper acc',upper_acc*100.0/vis)
#pkl.dump(total,open('gqa_t5_pred/t5_generated_temp.pkl','wb'))

T5 generated with generated template
68167
Upper acc 63.95
Upper acc 62.675
Upper acc 61.45
Upper acc 61.1
Upper acc 60.97
Upper acc 60.975
Upper acc 61.15714285714286
Upper acc 61.15625
Upper acc 61.144444444444446
Upper acc 61.04
Upper acc 60.91818181818182
Upper acc 60.90416666666667
Upper acc 60.98461538461538
Upper acc 60.91428571428571
Upper acc 60.81666666666667
Upper acc 60.821875
Upper acc 60.794117647058826
Upper acc 60.87222222222222
Upper acc 60.873684210526314
Upper acc 60.8775
Upper acc 60.79761904761905
Upper acc 60.80227272727273
Upper acc 60.8304347826087
Upper acc 60.82083333333333
Upper acc 60.794
Upper acc 60.86153846153846
Upper acc 60.86296296296296
Upper acc 60.92321428571429
Upper acc 60.91034482758621
Upper acc 60.92166666666667
Upper acc 60.880645161290325
Upper acc 60.884375
Upper acc 60.89393939393939
Upper acc 60.891176470588235
Total: 68167 
	Upper acc 60.898968709199465


In [27]:
q_t='query'
vis=0
upper=0.0
showing=True
random.shuffle(val_based_qt[q_t])
acc=0.0
upper_acc=0.0
print ('T5 generated with generated template')
total={}
print(len(val_based_qt[q_t]))
for row in val_based_qt[q_t]:
    vis+=1
        
    answers=row['answer']
    ques=row['question']
    
    try:
        statement=generate_statement_gt(row['full_ans'],answers)
    except:
        statement=generate_statement_bart(temp_file[row['key']])
    if '<mask>' not in statement:
        statement=generate_statement_bart(temp_file[row['key']])
    
    statement=statement.replace('<mask>',
                                '<extra_id_0>')
    t5_pred=candidate_generator(statement)
    
    upper,in_list=verify_ans([row['answer']],t5_pred)
    if upper:
        upper_acc+=1
    total[row['key']]=t5_pred    
    """print (ques,answers,row['img'])
    print ('\t',statement)
    print ('\tMine: ',len(t5_pred),upper,in_list)
    print (
        t5_pred,
        '\n')"""
    if vis%2000==0:
        print ('Upper acc',upper_acc*100.0/vis)
print ('Total:',vis,'\n\tUpper acc',upper_acc*100.0/vis)
pkl.dump(total,open('gqa_t5_pred/t5_gt_temp.pkl','wb'))

T5 generated with generated template
68167
Upper acc 64.5
Upper acc 65.8
Upper acc 65.43333333333334
Upper acc 65.6625
Upper acc 65.81
Upper acc 65.71666666666667
Upper acc 65.70714285714286
Upper acc 65.61875
Upper acc 65.59444444444445
Upper acc 65.71
Upper acc 65.68636363636364
Upper acc 65.61666666666666
Upper acc 65.75769230769231
Upper acc 65.61785714285715
Upper acc 65.62333333333333
Upper acc 65.66875
Upper acc 65.75588235294117
Upper acc 65.70555555555555
Upper acc 65.69473684210526
Upper acc 65.66
Upper acc 65.6452380952381
Upper acc 65.64772727272727
Upper acc 65.63260869565218
Upper acc 65.67083333333333
Upper acc 65.654
Upper acc 65.63269230769231
Upper acc 65.64814814814815
Upper acc 65.67142857142858
Upper acc 65.68448275862069
Upper acc 65.68666666666667
Upper acc 65.66451612903226
Upper acc 65.6125
Upper acc 65.58181818181818
Upper acc 65.61764705882354
Total: 68167 
	Upper acc 65.62559596285593


In [54]:
q_t='query'
vis=0
upper=0.0
showing=True
print ('Using bart')
print ('Using ground truth template')
#random.shuffle(val_based_qt[q_t])
acc=0.0
upper_acc=0.0
total={}
flag=True
print(len(val_based_qt[q_t]))
for row in val_based_qt[q_t]:
    vis+=1
    answers=row['answer']
    ques=row['question']
    try:
        statement=generate_statement_gt(row['full_ans'],answers)
    except:
        statement=generate_statement_bart(temp_file[row['key']])
    if '<mask>' not in statement:
        statement=generate_statement_bart(temp_file[row['key']])
    
    t5_pred=candidate_generator_bart(statement)
    
    upper,in_list=verify_ans([row['answer']],t5_pred)
    if upper:
        upper_acc+=1
        
    """print (ques,answers,row['img'])
    print ('\t',row['full_ans'].lower().replace(row['answer'],'<mask>'))
    print ('\t',statement)
    print ('\tMine: ',len(t5_pred),upper,in_list)
    print (
        t5_pred,
        '\n')"""
    total[row['key']]=t5_pred
    if vis%2000==0:
        print ('Upper acc',upper_acc*100.0/vis)
print ('Total:',vis,'\n\tUpper acc',upper_acc*100.0/vis)
pkl.dump(total,open('gqa_t5_pred/bart_gt_temp.pkl','wb'))

Using bart
Using ground truth template
68167
Upper acc 51.4
Upper acc 50.725
Upper acc 50.86666666666667
Upper acc 49.9875
Upper acc 49.86
Upper acc 49.891666666666666
Upper acc 49.99285714285714
Upper acc 49.70625
Upper acc 49.65555555555556
Upper acc 49.735
Upper acc 49.872727272727275
Upper acc 49.925
Upper acc 49.86153846153846
Upper acc 49.714285714285715
Upper acc 49.79
Upper acc 49.878125
Upper acc 49.88823529411765
Upper acc 49.9
Upper acc 49.95
Upper acc 49.9875
Upper acc 49.99047619047619
Upper acc 49.89772727272727
Upper acc 49.93478260869565
Upper acc 49.927083333333336
Upper acc 49.892
Upper acc 49.80384615384615
Upper acc 49.803703703703704
Upper acc 49.75535714285714
Upper acc 49.72586206896552
Upper acc 49.735
Upper acc 49.730645161290326
Upper acc 49.8046875
Upper acc 49.8
Upper acc 49.826470588235296
Total: 68167 
	Upper acc 49.82909619023868


In [55]:
q_t='query'
vis=0
upper=0.0
showing=True
print ('Using bart')
print ('Using generated template')
random.shuffle(val_based_qt[q_t])
acc=0.0
upper_acc=0.0
total={}
print(len(val_based_qt[q_t]))
for row in val_based_qt[q_t]:
    vis+=1
    
    answers=row['answer']
    ques=row['question']
    """try:
        statement=row['full_ans'][:-1].lower().replace(row['answer'],'<mask>')
    except:
        statement=generate_statement_bart(temp_file[row['key']])
    if '<mask>' not in statement:
        statement=generate_statement_bart(temp_file[row['key']])"""
    #print ('\t',statement)
    statement=generate_statement_bart(temp_file[row['key']])
    t5_pred=candidate_generator_bart(statement)
    
    upper,in_list=verify_ans([row['answer']],t5_pred)
    if upper:
        upper_acc+=1
        
    """print (ques,answers,row['img'])
    print ('\t',row['full_ans'].lower().replace(row['answer'],'<mask>'))
    print ('\t',statement)
    print ('\tMine: ',len(t5_pred),upper,in_list)
    print (
        t5_pred,
        '\n')"""
    total[row['key']]=t5_pred
    if vis%2000==0:
        print ('Upper acc',upper_acc*100.0/vis)
print ('Total:',vis,'\n\tUpper acc',upper_acc*100.0/vis)
pkl.dump(total,open('gqa_t5_pred/bart_generated_temp.pkl','wb'))

Using bart
Using generated template
68167
Upper acc 47.65
Upper acc 47.85
Upper acc 47.416666666666664
Upper acc 47.375
Upper acc 47.42
Upper acc 47.75833333333333
Upper acc 47.871428571428574
Upper acc 47.7875
Upper acc 47.56666666666667
Upper acc 47.56
Upper acc 47.413636363636364
Upper acc 47.34166666666667
Upper acc 47.41153846153846
Upper acc 47.38214285714286
Upper acc 47.46666666666667
Upper acc 47.4375
Upper acc 47.370588235294115
Upper acc 47.57222222222222
Upper acc 47.48684210526316
Upper acc 47.5825
Upper acc 47.58571428571429
Upper acc 47.472727272727276
Upper acc 47.54347826086956
Upper acc 47.55416666666667
Upper acc 47.618
Upper acc 47.59230769230769
Upper acc 47.6462962962963
Upper acc 47.6375
Upper acc 47.601724137931036
Upper acc 47.696666666666665
Upper acc 47.73548387096774
Upper acc 47.7515625
Upper acc 47.7469696969697
Upper acc 47.78235294117647
Total: 68167 
	Upper acc 47.787052386051904
